In [1]:
import melanoma as mel

%matplotlib inline
%load_ext autoreload
%autoreload -p 2

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")

import os
from string import Template

rootpath = '/hpcstor6/scratch01/s/sanghyuk.kim001'
# img_size = (224, 224) # height, width
# img_size = (150, 150) # height, width
utilInstance = mel.Util(rootpath)

2024-02-14 14:31:15.092110: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
SLURM_DIR = './SLURMS'
# if os.path.isExist(SLURM_DIR) is not True:
#     os.makedirs(SLURM_DIR)



SLURM_TEMPLATE = Template('''#!/bin/bash
#SBATCH --job-name=${DBname}_${classifier}_${img_height}h_${img_width}w
#SBATCH -p haehn -q haehn_unlim
#SBATCH -w chimera13
#SBATCH -n 2 # Number of cores
#SBATCH -N 1 # Ensure that all cores are on one machine
#SBATCH --gres=gpu:A100:1
#SBATCH --mem=$memory
#SBATCH -t 3-00:00
#SBATCH --open-mode=append
#SBATCH -o /home/sanghyuk.kim001/MELANOMA/melanoma-detection-CNN/SLURMS/LOGS/%x_%A_%a.out
#SBATCH -e /home/sanghyuk.kim001/MELANOMA/melanoma-detection-CNN/SLURMS/LOGS/%x_%A_%a.err
#SBATCH --array=1
##. /etc/profile,


echo `date`

eval "$$(conda shell.bash hook)"
conda activate clean_chimera_env

# For debugging purposes.
python --version
nvcc -V

# Print this sub-job's task ID
echo "My SLURM_ARRAY_TASK_ID: " $SLURM_ARRAY_TASK_ID

cd /home/sanghyuk.kim001/MELANOMA/melanoma-detection-CNN/

python train.py --DB $db --IMG_SIZE ${img_height} ${img_width} --CLASSIFIER $classifier --JOB_INDEX $SLURM_ARRAY_TASK_ID

# end
exit 0;
''')

In [3]:
DBs = [db.name for db in mel.DatasetType]
Classifiers = [c.name for c in mel.NetworkType]

combinedDBs = {
  'HAM10000+ISIC2016': [mel.DatasetType.HAM10000.name, mel.DatasetType.ISIC2016.name],
  'HAM10000+ISIC2017': [mel.DatasetType.HAM10000.name, mel.DatasetType.ISIC2017.name],
  'HAM10000+ISIC2018': [mel.DatasetType.HAM10000.name, mel.DatasetType.ISIC2018.name],
  'HAM10000+ISIC2019': [mel.DatasetType.HAM10000.name, mel.DatasetType.ISIC2019.name],
  'HAM10000+ISIC2020': [mel.DatasetType.HAM10000.name, mel.DatasetType.ISIC2020.name],
  'HAM10000+PH2': [mel.DatasetType.HAM10000.name, mel.DatasetType.PH2.name],
  'HAM10000+7point': [mel.DatasetType.HAM10000.name, mel.DatasetType._7_point_criteria.name],
  'HAM10000+PAD_UFES_20': [mel.DatasetType.HAM10000.name, mel.DatasetType.PAD_UFES_20.name],
  'HAM10000+MEDNODE': [mel.DatasetType.HAM10000.name, mel.DatasetType.MEDNODE.name],
  'HAM10000+KaggleMB': [mel.DatasetType.HAM10000.name, mel.DatasetType.KaggleMB.name],

}

# HAM10000 = 1
# ISIC2016= 2
# ISIC2017=3
# ISIC2018 = 4
# ISIC2019 = 5
# ISIC2020 = 6
# PH2 = 7
# _7_point_criteria = 8
# PAD_UFES_20 = 9
# MEDNODE = 10
# KaggleMB = 11


img_size = (150, 150)


# Single DB
# for d in DBs:
#   for c in Classifiers:
#     new_slurm = SLURM_TEMPLATE.substitute(db=[d], memory=32000, img_size=img_size, classifier=c, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
#     slurm_file = os.path.join(SLURM_DIR, d+'_'+c+'.sh')
#     with open(slurm_file, 'w') as f:
#       f.write(new_slurm)

# if len(combinedDBs) == 1:
#   DBname = DB
# elif len(combinedDBs) > 1:
#   DBname = '+'.join(DB)

# Combined DBs
for comb_d in combinedDBs:
  for c in Classifiers:
    DBname = '+'.join(combinedDBs[comb_d])
    new_slurm = SLURM_TEMPLATE.substitute(db=' '.join(combinedDBs[comb_d]), DBname=DBname, img_height=img_size[0], img_width=img_size[1], memory=32000, classifier=c, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
    combinedDB_name = '+'.join(combinedDBs[comb_d])
    slurm_file = os.path.join(SLURM_DIR, combinedDB_name+'_'+c+'.sh')
    with open(slurm_file, 'w') as f:
      f.write(new_slurm)

# slurm_file = os.path.join(SLURM_DIR, DBtemp[0]+'_'+Classifiers_temp[0]+'.sh')
# new_slurm = SLURM_TEMPLATE.substitute(db=d, memory=32000, classifier=c, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
# with open(os.path.join(SLURM_DIR, db+'_'+c+'.sbatch'), 'w') as f:
#   f.write(new_slurm)



In [4]:

# import subprocess
# subprocess.call(['sh', f'./{slurm_file}'])

import itertools

import os
batches = list(itertools.chain.from_iterable([glob.glob(f'{SLURM_DIR}/*.sh', recursive=True) for db in DB]))
for b in batches:
    os.system(f'sbatch {b}')


NameError: name 'itertools' is not defined

In [48]:
import melanoma as mel

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet101, ResNet152
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.efficientnet \
    import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, \
        EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
# from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0, EfficientNetV2B1, EfficientNetV2B2, \
#         EfficientNetV2B3, EfficientNetV2S, EfficientNetV2M, EfficientNetV2L
from tensorflow.keras.applications.resnet_v2 \
    import ResNet50V2, ResNet101V2, ResNet152V2
# from tensorflow.keras.applications.resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, \
    DenseNet201
from tensorflow.keras.applications.nasnet import NASNetMobile, NASNetLarge
# from tensorflow.keras.applications.convnext import ConvNeXtTiny, ConvNeXtSmall, \
#     ConvNeXtBase, ConvNeXtLarge, ConvNeXtXLarge

import logging
import sys
from pathlib import Path
import itertools
import glob
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")


DB = ['HAM10000', 'PH2'] # HAM10000, ISIC2016
IMG_SIZE = 150 # (150, 150)
CLASSIFIER = 'Xception' # 'ResNet50, VGG16'
JOB_INDEX = 11

if len(DB) == 1:
	DBname = DB
elif len(DB) > 1:
	DBname = '+'.join(DB)

rootpath = '/hpcstor6/scratch01/s/sanghyuk.kim001'
# img_size = (224, 224) # height, width
img_size = (IMG_SIZE, IMG_SIZE) # height, width
utilInstance = mel.Util(rootpath, img_size)

from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.keras.callbacks import EarlyStopping

img_height, img_width = utilInstance.getImgSize()

optimizer1 = Adam(lr=0.001)
optimizer2 = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
red_lr= ReduceLROnPlateau(monitor='val_accuracy', patience=3 , verbose=1, factor=0.7)
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 20)

CFG = dict(
			batch_size            =  64,   # 8; 16; 32; 64; bigger batch size => moemry allocation issue
			epochs                =  20,   # 5; 10; 20;
			last_trainable_layers =   0,
			verbose               =   1,   # 0; 1
			fontsize              =  14,
			num_classes           =  2, # binary

			# Images sizes
			img_height = img_height,   # Original: (450h, 600w)
            img_width = img_width,

			# Images augs
			ROTATION_RANGE        =   90.0,
			ZOOM_RANGE            =   0.1,
			HSHIFT_RANGE          =   0.1, # randomly shift images horizontally
			WSHIFT_RANGE          =   0.1, # randomly shift images vertically
			HFLIP                 = False, # randomly flip images
			VFLIP                 = False, # randomly flip images

			# Model settings
			pretrained_weights = 'imagenet',
			model_optimizer = optimizer2,
			# loss='binary_crossentropy',
			loss='categorical_crossentropy',
			metrics=['accuracy'],
			callbacks = [],

			# Postprocessing
			stopper_patience      =  0,   # 0.01; 0.05; 0.1; 0.2;
			# run_functions_eagerly = False,
            
            # save
            snapshot_path = '/hpcstor6/scratch01/s/sanghyuk.kim001/snapshot',
			experiment = f'{DBname}_{CLASSIFIER}_{IMG_SIZE}h_{IMG_SIZE}w_{JOB_INDEX}'
		)
base_model = mel.CNN(CFG=CFG)

classifierDict = {
            mel.NetworkType.ResNet50.name: ResNet50,
            mel.NetworkType.ResNet101.name: ResNet101,
            mel.NetworkType.ResNet152.name: ResNet152,
            mel.NetworkType.Xception.name: Xception,
            mel.NetworkType.InceptionV3.name: InceptionV3,
            mel.NetworkType.VGG16.name: VGG16,
            mel.NetworkType.VGG19.name: VGG19,
            mel.NetworkType.EfficientNetB0.name: EfficientNetB0,
            mel.NetworkType.EfficientNetB1.name: EfficientNetB1,
            mel.NetworkType.EfficientNetB2.name: EfficientNetB2,
            mel.NetworkType.EfficientNetB3.name: EfficientNetB3,
            mel.NetworkType.EfficientNetB4.name: EfficientNetB4,
            mel.NetworkType.EfficientNetB5.name: EfficientNetB5,
            mel.NetworkType.EfficientNetB6.name: EfficientNetB6,
            mel.NetworkType.EfficientNetB7.name: EfficientNetB7,

            mel.NetworkType.ResNet50V2.name: ResNet50V2,
            mel.NetworkType.ResNet101V2.name: ResNet101V2,
            mel.NetworkType.ResNet152V2.name: ResNet152V2,

            mel.NetworkType.MobileNet.name: MobileNet,
            mel.NetworkType.MobileNetV2.name: MobileNetV2,

            mel.NetworkType.DenseNet121.name: DenseNet121,
            mel.NetworkType.DenseNet169.name: DenseNet169,
            mel.NetworkType.DenseNet201.name: DenseNet201,

            mel.NetworkType.NASNetMobile.name: NASNetMobile,
            mel.NetworkType.NASNetLarge.name: NASNetLarge,
        }

# Training DBs with Networks
dbpath = f'/hpcstor6/scratch01/s/sanghyuk.kim001/melanomaDB/customDB/{CLASSIFIER}'
# picklename = f'{DB}_{IMG_SIZE}h_{IMG_SIZE}w_binary'
del_augmentation = {'ROTATION_RANGE':0.0, 'ZOOM_RANGE':0.0, 'HSHIFT_RANGE':0.0, 'WSHIFT_RANGE':0.0}
CFG.update(del_augmentation)

ori_pkl = list(itertools.chain.from_iterable([glob.glob(f'{dbpath}/{db}_{IMG_SIZE}h_{IMG_SIZE}w*', recursive=True) for db in DB]))
aug_pkl = list(itertools.chain.from_iterable([glob.glob(f'{dbpath}/{db}_augmentedWith_*Melanoma_*Non-Melanoma_{IMG_SIZE}h_{IMG_SIZE}w*', recursive=True) for db in DB]))


if len(DB) == 1:
	trainimages, testimages, validationimages, \
				trainlabels, testlabels, validationlabels, num_classes\
					= utilInstance.loadDatasetFromFile(dbpath+'/'+Path(ori_pkl).stem+'.pkl')
elif len(DB) > 1:
	trainimages, testimages, validationimages, \
				trainlabels, testlabels, validationlabels, \
					= utilInstance.combineDatasets(aug_pkl)


model_name = CFG['experiment']
model = base_model.transformer(classifierDict[CLASSIFIER])
# model = base_model.inceptionV3()

history = base_model.fit_model(
    model = model,
    model_name = model_name,
    trainimages = trainimages,
    trainlabels = trainlabels,
    validationimages = validationimages,
    validationlabels = validationlabels,
)

visualizer = mel.Visualizer()
visualizer.visualize_model(model = model, plot_path=CFG['snapshot_path'], model_name = model_name)

visualizer.visualize_performance(
    model_name = model_name,
    history = history
)


/home/sanghyuk.kim001/anaconda3/envs/clean_chimera_env/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Combining...
Combining 1 db out of 1 dbs
Combining 2 db out of 1 dbs
Stacking training images
Stacking training labels
Stacking validation images
Stacking validation labels
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 2048)              20861480  
_________________________________________________________________
flatten_4 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_388 (Bat (None, 512)               2048      
_________________________________________________________________
dense_13 (Den

KeyboardInterrupt: 